<a href="https://colab.research.google.com/github/hamiq/sars_inhibitor_prediction/blob/main/Data_Collection_and_Pre_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Collection and Pre-Processing**

---

## **ChEMBL Database**

The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/) is a database that contains curated bioactivity data of more than 2 million compounds.

The ChEMBL web service package allows retrieval of bioactivity data from the database.

In [ ]:
! pip install chembl_webresource_client

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 596 kB 19.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires itsdangerous<2.0,>=0.24, but you have itsdangerous 2.0.1 which is incompatible.


In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

## **Search for target protein**

In [ ]:
# Target search for coronavirus, same action as if going to ChEMBL and searching for protein in the search bar
target = new_client.target
target_query = target.search('coronavirus')
targets = pd.DataFrame.from_dict(target_query)
targets

cross_references  ...   tax_id
0                                                 []  ...    11119
1                                                 []  ...   227859
2                                                 []  ...    12663
3                                                 []  ...    11137
4  [{'xref_id': 'P0C6U8', 'xref_name': None, 'xre...  ...   227859
5                                                 []  ...  1335626
6  [{'xref_id': 'P0C6X7', 'xref_name': None, 'xre...  ...   227859
7                                                 []  ...  2697049

[8 rows x 9 columns]

The target protein for which drug candidates will be analyzed is the *SARS coronavirus 3C-like proteinase*.

In [ ]:
selected_target = targets.target_chembl_id[4]
selected_target

'CHEMBL3927'

To make the data cleaner, only one measurement for bioactivity will be included: IC$_{50}$ values in nM (nanomolar) unit. IC$_{50}$ is half-maximal inhibitory concentration, meaning the amount of the drug necessary to inhibit 50% of the biological activity, and it is the most widely used measure of drug efficacy.

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(res)

In [ ]:
df.head(3)

activity_comment  activity_id  ... upper_value value
0             None      1480935  ...        None   7.2
1             None      1480936  ...        None   9.4
2             None      1481061  ...        None  13.5

[3 rows x 45 columns]

In [ ]:
df.standard_type.unique()

array(['IC50'], dtype=object)

## **Save the pulled data**

In [ ]:
df.to_csv('bioactivity_data.csv', index=False)

In [ ]:
! head bioactivity_data.csv

activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
,1480935,[],CHEMBL829584,In vitro inhibitory concentration against SARS coronavirus main protease (SARS CoV 3C-like protease),B,,,BAO_0000190,BAO_0000357,single protein format,Cc1noc(C)c1CN1C(=O)C(=O)c2cc(C#N)ccc21,,,CHEMBL1139624,Bioorg. Med. Chem. Lett.,2005,"{'bei': '18.28', 'le': '0.33', 'lle': '3.25', 'sei': '5.90'}

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)


Mounted at /content/gdrive/


In [ ]:
! mkdir "/content/gdrive/My Drive/Colab Notebooks/SARS Bioactivity"

mkdir: cannot create directory ‘/content/gdrive/My Drive/Colab Notebooks/SARS Bioactivity’: File exists


In [ ]:
! cp bioactivity_data.csv "/content/gdrive/My Drive/Colab Notebooks/SARS Bioactivity"

In [ ]:
! ls -l "/content/gdrive/My Drive/Colab Notebooks/SARS Bioactivity"

total 83169
-rw------- 1 root root    70334 Nov  7 01:38  bioactivity_data.csv
-rw------- 1 root root     9337 Nov  7 00:31  bioactivity_preprocessed_data.csv
-rw------- 1 root root    28228 Nov  7 01:38 'Data Collection and Pre-Processing.ipynb'
-rw------- 1 root root 85055499 Mar 11  2020  Miniconda3-py37_4.8.2-Linux-x86_64.sh


## **Data pre-processing**
If any compounds have a missing value for the **standard_value** column, they should be dropped as this is the measurement that the prediction model will be based off of.

In [ ]:
df2 = df[df.standard_value.notna()]
df2

activity_comment  activity_id  ... upper_value  value
0               None      1480935  ...        None    7.2
1               None      1480936  ...        None    9.4
2               None      1481061  ...        None   13.5
3               None      1481065  ...        None  13.11
4               None      1481066  ...        None    2.0
..               ...          ...  ...         ...    ...
128             None     12041507  ...        None   10.6
129             None     12041508  ...        None   10.1
130             None     12041509  ...        None   11.5
131             None     12041510  ...        None   10.7
132             None     12041511  ...        None   78.9

[133 rows x 45 columns]

### **Labeling compounds as either being active, inactive or intermediate**
Compounds having values for IC50 that are less than 1000 nM will be considered to be **active** while those greater than 10,000 nM will be considered to be **inactive**. Values between 1,000 and 10,000 nM will be considered **intermediate**. 

In [ ]:
bioactivity_class = []
for i in df2.standard_value:
  if float(i) >= 10000:
    bioactivity_class.append("inactive")
  elif float(i) <= 1000:
    bioactivity_class.append("active")
  else:
    bioactivity_class.append("intermediate")

### **Create a dataframe of the relevant data points**

In [ ]:
# chembl ids
mol_cid = []
for i in df2.molecule_chembl_id:
  mol_cid.append(i)

# canonical smiles
canonical_smiles = []
for i in df2.canonical_smiles:
  canonical_smiles.append(i)

# IC50 values
standard_value = []
for i in df2.standard_value:
  standard_value.append(i)

data_tuples = list(zip(mol_cid, canonical_smiles, bioactivity_class, standard_value))
df3 = pd.DataFrame(data_tuples, columns=['molecule_chembl_id', 'canonical_smiles', 'bioactivity_class', 'standard_value'])

In [ ]:
df3

molecule_chembl_id  ... standard_value
0         CHEMBL187579  ...         7200.0
1         CHEMBL188487  ...         9400.0
2         CHEMBL185698  ...        13500.0
3         CHEMBL426082  ...        13110.0
4         CHEMBL187717  ...         2000.0
..                 ...  ...            ...
128      CHEMBL2146517  ...        10600.0
129       CHEMBL187460  ...        10100.0
130       CHEMBL363535  ...        11500.0
131       CHEMBL227075  ...        10700.0
132        CHEMBL45830  ...        78900.0

[133 rows x 4 columns]

In [ ]:
df3.to_csv('bioactivity_preprocessed_data.csv', index=False)

In [ ]:
! cp bioactivity_preprocessed_data.csv "/content/gdrive/My Drive/Colab Notebooks/SARS Bioactivity"